In [ ]:
### Import Libraries.

library("DCATS")
library("Seurat")
library("dplyr")
library("qs")
library("ggplot2")

In [ ]:
### Load Seurat.

setwd("/folder/")
Seurat <- qread("Seurat.qs")

In [ ]:
### Function to Prepare Count Matrix and Metadata.

prepare_dcats_input <- function(seurat_obj, cluster_col, sample_col = "Sample_ID", status_col = "Status") {
    
  stopifnot(cluster_col %in% colnames(seurat_obj@meta.data))
  stopifnot(sample_col %in% colnames(seurat_obj@meta.data))
  stopifnot(status_col %in% colnames(seurat_obj@meta.data))
  
  seurat_obj@meta.data[[ "Sample_ID_Status" ]] <- paste0(seurat_obj@meta.data[[sample_col]], "_", seurat_obj@meta.data[[status_col]])
  count_mat <- table(seurat_obj@meta.data[[ "Sample_ID_Status" ]], seurat_obj@meta.data[[ cluster_col ]])
  count_mat <- as.matrix(count_mat)
  
  rownames_sorted <- rownames(count_mat)[order(rownames(count_mat))]
  count_mat <- count_mat[rownames_sorted, , drop = FALSE]
  
  meta_df <- data.frame(Sample_ID_Status = rownames(count_mat), stringsAsFactors = FALSE)
    
  tmp <- do.call(rbind, strsplit(meta_df$Sample_ID_Status, "_", fixed = TRUE))
  colnames(tmp) <- c(sample_col, status_col)
  meta_df[[sample_col]] <- tmp[,1]
  meta_df[[status_col]]  <- tmp[,2]
  rownames(meta_df) <- meta_df$Sample_ID_Status
  
  list(count_mat = count_mat, meta_df = meta_df)
}

In [ ]:
### Example.

data_ex <- prepare_dcats_input(Seurat, cluster_col = "Cluster_Column")
count_mat_ex <- data_ex$count_mat
meta_ex <- data_ex$meta_df

In [ ]:
### Filter Rows for Status in control, C9ALS, sALS.

keep_rows <- meta_ex$Status %in% c("Control", "sALS")
count_mat_ex <- count_mat_ex[keep_rows, , drop = FALSE]
meta_ex <- meta_ex[keep_rows, , drop = FALSE]

In [ ]:
### Similarity Matrix.

if (!is.null(SeuratObject_SC@graphs$RNA_nn)) {
  knn_adj <- SeuratObject_SC@graphs$RNA_nn
  # If RNA_nn is sparse adjacency of cells, but DCATS expects a cluster-level similarity,
  # use their helper knn_simMat when provided a graph for clusters (examples vary by usage).
  # We'll attempt to compute cluster-level knn via subsetting to the clusters used above:
  # build a cell -> cluster mapping
  cell2cluster <- SeuratObject_SC@meta.data$Subclusters_Activation
  names(cell2cluster) <- rownames(SeuratObject_SC@meta.data)
  # NOTE: knn_simMat expects a kNN matrix and labels; see DCATS vignette for appropriate use.
  # If you want cluster-level similarity, try:
  knn_mat <- knn_simMat(knn = SeuratObject_SC@graphs$RNA_nn, labels = SeuratObject_SC@meta.data$Subclusters_Activation)
} else {
  # If no graph, fallback to identity (no smoothing). This is acceptable but less powerful.
  knn_mat <- diag(ncol(count_mat_sals))
  colnames(knn_mat) <- colnames(count_mat_sals)
  rownames(knn_mat) <- colnames(count_mat_sals)
}

In [ ]:
### DCATS (GLM).

res <- dcats_GLM(count_mat = count_mat_sals, design_mat = meta_sals,
                      similarity_mat = knn_mat, model_type = "GLM")
print(head(res$coef))

In [ ]:
### Save Results.

setwd("/folder/")
qsave(res, "dcats_res.qs")